In [ ]:
# Clear environment
rm(list = ls())

if(!require(dplyr)) install.packages("dplyr")
if(!require(randomForest)) install.packages("randomForest")
if(!require(gbm)) install.packages("gbm")
if(!require(gam)) install.packages("gam")
if(!require(caret)) install.packages("caret")
if(!require(e1071)) install.packages("e1071")
if(!require(ROCR)) install.packages("ROCR")
if(!require(themis)) install.packages("themis") 
if(!require(patchwork)) install.packages("patchwork")
# if (!requireNamespace("ROCR", quietly = TRUE)) {install.packages("ROCR")}

update.packages(c("themis", "recipes"))

library('dplyr');

#Read the data
## please change the file location
# setwd('~/Cathy/OMS/Data Mining&Stat Learn - ISYE-7406-OAN/Project/Data');
setwd('~/Cathy/OMS/Data & Visual Analytics - CSE6242OAN O01 O3 AO/Project/data/cleaned_data');

Raw_Data <- read.csv(file = "cleaned_data_updated.csv", header=TRUE, dec=",");

# Check unique value of contributing factors
lapply(Raw_Data[c('CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2')], unique); 

# Function to group contributing factors 
group_factors <- function(factors) {
  factor_group <- sapply(factors, function(factor) {
    if (factor %in% c("Driver Inexperience", "Driver Inattention/Distraction", "Aggressive Driving/Road Rage", 
                      "Texting", "Cell Phone (hands-free)", "Eating or Drinking", "Passenger Distraction", "Fell Asleep", 
                      "Backing Unsafely", 
                      "Cell Phone (hand-Held)", "Using On Board Navigation Device", 
                      "Other Electronic Device", "Listening/Using Headphones", 
                      "Cell Phone (hand-held)")) {
      return("Driver Behavior/Inattention")
    } else if (factor %in% c("Brakes Defective", "Tire Failure/Inadequate", "Steering Failure", "Headlights Defective", 
                             "Windshield Inadequate", "Other Lighting Defects", "Tow Hitch Defective", "Oversized Vehicle", "Accelerator Defective", "Driverless/Runaway Vehicle", 
                             "Vehicle Vandalism", "Tinted Windows")) {
      return("Vehicle Condition/Mechanical Failure")
    } else if (factor %in% c("Traffic Control Disregarded", "Lane Marking Improper/Inadequate", "Traffic Control Device Improper/Non-Working")) {
      return("Traffic Control Issues")
    } else if (factor %in% c("Pavement Slippery", "Pavement Defective", "Shoulders Defective/Improper", "Obstruction/Debris")) {
      return("Road Conditions")
    } else if (factor %in% c("Glare", "View Obstructed/Limited")) {
      return("Environmental Factors")
    } else if (factor %in% c("Unsafe Speed", "Following Too Closely", "Passing Too Closely", "Passing or Lane Usage Improper", 
                             "Unsafe Lane Changing", "Failure to Yield Right-of-Way", "Failure to Keep Right", "Turning Improperly", "Oversized Vehicle", 
                             "Alcohol Involvement", "Drugs (illegal)", "Drugs (Illegal)", "Prescription Medication")) {
      return("Traffic Violations/Unsafe Driving")
    } else if (factor %in% c("Pedestrian/Bicyclist/Other Pedestrian Error/Confusion", "Reaction to Uninvolved Vehicle", 
                             "Reaction to Other Uninvolved Vehicle", "Animals Action", "Other Vehicular", "Outside Car Distraction")) {
      return("Pedestrian/Other Vehicle Involvement")
    } else if (factor %in% c("Unspecified", "MISSING", "80", "1")) {
      return("Unspecified or Missing")
    } else if (factor %in% c("Illness", "Illnes", "Lost Consciousness", "Physical Disability", "Fatigued/Drowsy")) {
      return("Health Issues")
    } else {
      return("Other")
    }
  })
  
  return(factor_group)
}

# # Filter rows where 'GROUPED_FACTOR_VEHICLE_1' is categorized as 'Other'
# other_vehicle_1 <- Raw_Data[Raw_Data$GROUPED_FACTOR_VEHICLE_1 == "Other", ]
# unique_values_vehicle_1 <- unique(other_vehicle_1$CONTRIBUTING_FACTOR_VEHICLE_1)
# 
# other_vehicle_2 <- Raw_Data[Raw_Data$GROUPED_FACTOR_VEHICLE_2 == "Other", ]
# unique_values_vehicle_2 <- unique(other_vehicle_2$CONTRIBUTING_FACTOR_VEHICLE_2)
# 
# unique_others <- unique(c(unique_values_vehicle_1, unique_values_vehicle_2))
# print(unique_others)

Raw_Data$GROUPED_FACTOR_VEHICLE_1 <- group_factors(Raw_Data$CONTRIBUTING_FACTOR_VEHICLE_1)
Raw_Data$GROUPED_FACTOR_VEHICLE_2 <- group_factors(Raw_Data$CONTRIBUTING_FACTOR_VEHICLE_2)

# lapply(Raw_Data[c('GROUPED_FACTOR_VEHICLE_1', 'GROUPED_FACTOR_VEHICLE_2')], unique); 

# Raw_Data %>%
#   group_by(GROUPED_FACTOR_VEHICLE_1) %>%
#   summarise(count = n());
# 
# Raw_Data %>%
#   group_by(GROUPED_FACTOR_VEHICLE_2) %>%
#   summarise(count = n());

# Convert data types

# Check if the "Data" data frame exists
if (exists("Data")) {
  # If it exists, remove it
  rm(Data)
  print("Data data frame dropped.")
} else {
  print("Data data frame does not exist.")
}


Data <- Raw_Data %>%
  mutate(
    NUMBER_OF_PERSONS_INJURED = as.numeric(NUMBER_OF_PERSONS_INJURED),
    NUMBER_OF_PERSONS_KILLED = as.numeric(NUMBER_OF_PERSONS_KILLED),
    LATITUDE_CRASH = as.numeric(LATITUDE_CRASH),
    LONGITUDE_CRASH = as.numeric(LONGITUDE_CRASH),
    PRCP = as.numeric(PRCP),
    SNOW = as.numeric(SNOW),
    SNWD = as.numeric(SNWD),
    TMAX = as.numeric(TMAX),
    TMIN = as.numeric(TMIN),
    BOROUGH = as.numeric(factor(BOROUGH)),  # Convert categorical to factor
    ZIP_CODE = as.numeric(factor(ZIP_CODE)),
    GROUPED_FACTOR_VEHICLE_1 = as.numeric(factor(GROUPED_FACTOR_VEHICLE_1)),
    GROUPED_FACTOR_VEHICLE_2 = as.numeric(factor(GROUPED_FACTOR_VEHICLE_2)),
    VEHICLE_1_TYPE_REGROUP = as.numeric(factor(VEHICLE_TYPE_CODE_1_REGROUP)),
    VEHICLE_2_TYPE_REGROUP = as.numeric(factor(VEHICLE_TYPE_CODE_2_REGROUP)),
    RUSH_HOUR = as.numeric(factor(RUSH_HOUR)),
    SEASON = as.numeric((factor(SEASON)))
  ) %>%
  select(-CONTRIBUTING_FACTOR_VEHICLE_1, -CONTRIBUTING_FACTOR_VEHICLE_2, -VEHICLE_TYPE_CODE_1_REGROUP, -VEHICLE_TYPE_CODE_2_REGROUP) 


# Create a dummy variable
Data$DUMMY_INJURED <- ifelse(Data$NUMBER_OF_PERSONS_INJURED > 0, 1, 0);
Data$DUMMY_KILLED <- ifelse(Data$NUMBER_OF_PERSONS_KILLED > 0, 1, 0);
Data <- Data %>% select(DUMMY_KILLED, DUMMY_INJURED, everything());
# head(Data);

# check unique value of dummy variables and their distribution
lapply(Data[c('DUMMY_KILLED', 'DUMMY_INJURED')], unique); 


Data %>%
  group_by(DUMMY_KILLED) %>%
  summarise(count = n());
# DUMMY_KILLED   count
# <dbl>   <int>
#  0 1648023
#  1    2425


Data %>%
  group_by(DUMMY_INJURED) %>%
  summarise(count = n());
# DUMMY_INJURED   count
# <dbl>   <int>
#  0 1247274
#  1  403174


# View the updated dataframe with new columns
head(Raw_Data)

########################EDA######################################################
#-------------------------------------------------------------------------------#
# EDA_Outliers 
# boxplot(Data$YEAR);
# boxplot.stats(Data$YEAR)$out;


# EDA correlation matrix
# colnames(Data)
Data[rowSums(is.na(Data)) > 0, ]

Cor_Data <- Data %>%
  select(-c('ON_STREET_NAME',
            'CROSS_STREET_NAME',
            'CRASH_DATETIME',
            'NUMBER_OF_PEDESTRIANS_INJURED', 
            'NUMBER_OF_PEDESTRIANS_KILLED',
            'NUMBER_OF_PERSONS_INJURED', 
            'NUMBER_OF_PERSONS_KILLED',
            'NUMBER_OF_CYCLIST_INJURED',
            'NUMBER_OF_CYCLIST_KILLED',
            'NUMBER_OF_MOTORIST_INJURED',
            'NUMBER_OF_MOTORIST_KILLED',
            'VEHICLE_TYPE_CODE_1',
            'VEHICLE_TYPE_CODE_2',
            'COLLISION_ID', 
            'STATION',
            'LATITUDE_WEATHER_STATION',
            'LONGITUDE_WEATHER_STATION',
            'DISTANCE_FROM_WEATHER_STATION',
            'NAME',                     
            'ELEVATION'))

dim(Cor_Data); 
# 1650448      22
head(Cor_Data); 

Cor_Data %>%
  group_by(DUMMY_KILLED) %>%
  summarise (Count=n())
# # A tibble: 2 × 2
# DUMMY_KILLED   Count
# <dbl>   <int>
# 0 1648023
# 1    2425

2425/(2425+1648023) # [1] 0.001469298

summary(Cor_Data)

# Scatter_Matrix

corr_data = cor(Cor_Data);
corr_data;
library(corrplot);
plot.new()
dev.off()
corrplot(corr_data, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(corr_data, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);
corrplot(corr_data, method="ellipse",tl.cex = 0.5, cl.cex = 0.5); 

# corrplot for dummy_killed vs. predictors
cor_data_Road = cor(Data %>% select(c('DUMMY_KILLED','RUSH_HOUR','BOROUGH', 'ZIP_CODE')));
corrplot(cor_data_Road, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Road, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_VEHICLE = cor(Data %>% select(c('DUMMY_KILLED','RUSH_HOUR','VEHICLE_TYPE_CODE_1_REGROUP','VEHICLE_TYPE_CODE_1_REGROUP','BOROUGH')));
corrplot(cor_data_VEHICLE, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_VEHICLE, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_Weather = cor(Data %>% select(c('DUMMY_KILLED','RUSH_HOUR','SEASON','PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN')));
corrplot(cor_data_Weather, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Weather, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_Road_Weather = cor(Data %>% select(c('DUMMY_KILLED','RUSH_HOUR','SEASON','VEHICLE_TYPE_CODE_1_REGROUP','VEHICLE_TYPE_CODE_1_REGROUP','BOROUGH', 'ZIP_CODE','PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN')));
corrplot(cor_data_Road_Weather, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Road_Weather, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

# corrplot for dummy_injured vs. predictors
cor_data_Road2 = cor(Data %>% select(c('DUMMY_INJURED','RUSH_HOUR','BOROUGH', 'ZIP_CODE')));
corrplot(cor_data_Road2, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Road2, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_VEHICLE2 = cor(Data %>% select(c('DUMMY_INJURED','RUSH_HOUR','VEHICLE_TYPE_CODE_1_REGROUP','VEHICLE_TYPE_CODE_1_REGROUP','BOROUGH')));
corrplot(cor_data_VEHICLE2, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_VEHICLE2, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_Weather2 = cor(Data %>% select(c('DUMMY_INJURED','RUSH_HOUR','SEASON','PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN')));
corrplot(cor_data_Weather2, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Weather2, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

cor_data_Road_Weather2 = cor(Data %>% select(c('DUMMY_INJURED','RUSH_HOUR','SEASON','VEHICLE_TYPE_CODE_1_REGROUP','VEHICLE_TYPE_CODE_1_REGROUP','BOROUGH', 'ZIP_CODE','PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN')));
corrplot(cor_data_Road_Weather2, method="color", tl.cex = 0.5, cl.cex = 0.5);
corrplot(cor_data_Road_Weather2, method="number", tl.cex = 0.5, number.cex = 0.5, cl.cex = 0.5);

# boxplots
# head(Data)
library(ggplot2)
library(GGally)

font_size <- 10 # Set your desired font size

for (i in 2:dim(Cor_Data)[2]) {
  print(
    ggplot(Cor_Data, aes(x = as.factor(DUMMY_KILLED), y = Cor_Data[, i])) +
      geom_boxplot() +
      labs(y = colnames(Cor_Data)[i]) +
      theme(
        axis.title = element_text(size = font_size),
        axis.text = element_text(size = font_size),
        plot.title = element_text(size = font_size),
        text = element_text(size = font_size)
      )
  )
}
# According to boxplots, the following predictors have different distribution for dummy_killed 1 vs. 0: 
# significant difference: BOROUGH, RUSH_HOUR, YEAR, VEHICLE_2_TYPE_REGROUP, HOUR
# Moderate difference: LATITUDE_CRASH, GROUPED_FACTOR_VEHICLE_1, SNOW, SNWD, PCRP, TMIN, HOUR
# Minimum difference: ZIP_CODE, lONGTITUDE_CRASH, TMAX, MONTH, DAY, VEHICLE_1_TYPE_REGROUP, SEASON, GROUPED_FACTOR_VEHICLE_2, WEEKDAY

##-------------------------------------------------------------------------##
######################END of EDA#############################################

##############Prepare datasets for modeling##################################
#---------------------------------------------------------------------------#

# Load necessary libraries
library(smotefamily)
library(ROCR)
library(caret)  # For confusionMatrix

# Define helper functions for repeated operations
get_sample_data <- function(data, prop = 0.10, seed = 123) {
  set.seed(seed)
  n_rows <- nrow(data)
  random_indices <- sample(1:n_rows, floor(prop * n_rows), replace = FALSE)
  return(data[random_indices, ])
}

# Clean and preprocess the data
Data1 <- Cor_Data
dim(Data1)  # [1] 1650448 22

# Subset the data, remove multicollinear or irrelevant columns
Data2 <- subset(Cor_Data, select = c(DUMMY_KILLED, DUMMY_INJURED, BOROUGH, RUSH_HOUR, YEAR,
                                     VEHICLE_1_TYPE_REGROUP, VEHICLE_2_TYPE_REGROUP,
                                     LATITUDE_CRASH, GROUPED_FACTOR_VEHICLE_1, SNOW, SNWD, PRCP, TMIN))
dim(Data2)  # [1] 1650448 13

# -------------------- Split Data into Training and Testing Sets --------------------
set.seed(6242)
n <- nrow(Data1)
n1 <- round(n * 0.20)  # 20% for testing

flag <- sort(sample(1:n, n1))
Data1_train <- Data1[-flag, ]
Data1_test <- Data1[flag, ]
Data2_train <- Data2[-flag, ]
Data2_test <- Data2[flag, ]

# Sample 10% of data for smaller subset processing
Data1_train_10 <- get_sample_data(Data1_train)
Data1_test_10 <- get_sample_data(Data1_test)
Data2_train_10 <- get_sample_data(Data2_train)
Data2_test_10 <- get_sample_data(Data2_test)

# Remove the highly correlated column (column 2 DUMMY_INJURED)
remove_column <- function(data) {
  data[, -2]
}

Data1_train_processed <- remove_column(Data1_train)
Data1_test_processed <- remove_column(Data1_test)
Data1_train_10_processed <- remove_column(Data1_train_10)
Data1_test_10_processed <- remove_column(Data1_test_10)
Data2_train_processed <- remove_column(Data2_train)
Data2_test_processed <- remove_column(Data2_test)
Data2_train_10_processed <- remove_column(Data2_train_10)
Data2_test_10_processed <- remove_column(Data2_test_10)

# Extract true response values
y1 <- Data1_train$DUMMY_KILLED
y2 <- Data1_test$DUMMY_KILLED
y3 <- Data2_train$DUMMY_KILLED
y4 <- Data2_test$DUMMY_KILLED

y1_10 <- Data1_train_10$DUMMY_KILLED
y2_10 <- Data1_test_10$DUMMY_KILLED
y3_10 <- Data2_train_10$DUMMY_KILLED
y4_10 <- Data2_test_10$DUMMY_KILLED

# -------------------- Function to Calculate F1 Score --------------------
F1_Score <- function(predicted, actual) {
  tp <- sum(predicted == 1 & actual == 1)
  fp <- sum(predicted == 1 & actual == 0)
  fn <- sum(predicted == 0 & actual == 1)
  
  # Check for zero division errors and handle them
  precision <- ifelse((tp + fp) == 0, 0, tp / (tp + fp))
  recall <- ifelse((tp + fn) == 0, 0, tp / (tp + fn))
  
  # Handle edge case where precision and recall are both zero
  f1 <- ifelse((precision + recall) == 0, 0, 2 * precision * recall / (precision + recall))
  
  return(f1)
}

# -------------------- Logistic Regression with SMOTE --------------------
logistic_regression_smote <- function(train_data, test_data, response_col = "DUMMY_KILLED") {
  # Separate features and target
  X_train <- train_data[, -which(names(train_data) == response_col)]
  y_train <- train_data[[response_col]]
  
  # Apply SMOTE
  smote_output <- SMOTE(X_train, y_train, K = 5)
  train_smote <- smote_output$data
  colnames(train_smote)[ncol(train_smote)] <- response_col
  train_smote[[response_col]] <- as.numeric(as.character(train_smote[[response_col]]))
  
  # Logistic Regression Model
  mod_glm_test <- glm(DUMMY_KILLED ~ ., family = binomial, data = train_smote, 
                      weights = ifelse(train_smote$DUMMY_KILLED == 1, 1, 10))
  
  # Define cutoff range
  cutoff_values <- seq(0.1, 0.9, 0.1)
  Train_Error_glm <- NULL
  Test_Error_glm <- NULL
  
  # Calculate Train and Test Errors for different cutoffs
  for (c in cutoff_values) {
    y1hat <- ifelse(predict(mod_glm_test, train_smote[, -which(names(train_smote) == response_col)], type = "response") < c, 0, 1)
    y2hat <- ifelse(predict(mod_glm_test, test_data[, -which(names(test_data) == response_col)], type = "response") < c, 0, 1)
    Train_Error_glm <- c(Train_Error_glm, mean(y1hat != train_smote$DUMMY_KILLED))
    Test_Error_glm <- c(Test_Error_glm, mean(y2hat != test_data[[response_col]]))
  }
  
  # Plot and evaluate errors
  plot(cutoff_values, Test_Error_glm)
  return(list(Train_Error_glm = Train_Error_glm, Test_Error_glm = Test_Error_glm))
}

# Apply Logistic Regression with SMOTE for Data1 and Data2
logistic_regression_smote(Data1_train_10_processed, Data1_test_10_processed, response_col = "DUMMY_KILLED")
# $Train_Error_glm
# [1] 0.3313495 0.4450501 0.4960949 0.5003509 0.4998540 0.4998009 0.4998009 0.4998009 0.4998009
# $Test_Error_glm
# [1] 0.346238905 0.069617377 0.011299948 0.002665940 0.001363265 0.001302675 0.001302675
# [8] 0.001302675 0.001302675

logistic_regression_smote(Data2_train_10_processed, Data2_test_10_processed, response_col = "DUMMY_KILLED")
# $Train_Error_glm
# [1] 0.3271884 0.4510699 0.4956056 0.4998502 0.4998009 0.4998009 0.4998009 0.4998009 0.4998009
# $Test_Error_glm
# [1] 0.362658669 0.067617922 0.006695144 0.001423854 0.001332970 0.001302675 0.001302675
# [8] 0.001302675 0.001302675

# -------------------- Logistic Regression with Stepwise Selection --------------------
# Logistic Regression with Stepwise Selection and SMOTE for imbalanced data
logistic_regression_step_smote <- function(train_data, test_data, response_col = "DUMMY_KILLED") {
  
  # Separate features and target
  X_train <- train_data[, -which(names(train_data) == response_col)]
  y_train <- train_data[[response_col]]
  
  # Apply SMOTE to balance the data
  smote_output <- SMOTE(X_train, y_train, K = 5)
  train_smote <- smote_output$data
  colnames(train_smote)[ncol(train_smote)] <- response_col
  train_smote[[response_col]] <- as.factor(train_smote[[response_col]])
  
  # Check NA values.
  # print(paste("NA values in train_smote:", sum(is.na(train_smote))))
  # print(paste("NA values in test_data:", sum(is.na(test_data))))
  
  # Fit the logistic regression model using glm (with weights for the imbalance)
  mod_glms_test <- glm(as.factor(DUMMY_KILLED) ~ ., family = binomial, data = train_smote,
                       weights = ifelse(train_smote$DUMMY_KILLED == 1, 1, 10))
  
  # Perform stepwise feature selection using base R's step() function
  mod_glms_test_step <- stats::step(mod_glms_test, direction = "both")
  
  # Define cutoff values for testing
  cutoff_values <- seq(0.1, 0.9, 0.1)
  Train_Error_glms <- NULL
  Test_Error_glms <- NULL
  
  # Calculate Train and Test Errors for different cutoff values
  for (c in cutoff_values) {
    # Train predictions
    train_preds <- predict(mod_glms_test_step, train_smote[, -which(names(train_smote) == response_col)], type = "response")
    y1hat <- ifelse(train_preds < c, 0, 1)
    
    # Test predictions
    test_preds <- predict(mod_glms_test_step, test_data[, -which(names(test_data) == response_col)], type = "response")
    y2hat <- ifelse(test_preds < c, 0, 1)
    
    # Store the error for the current cutoff value
    Train_Error_glms <- c(Train_Error_glms, mean(y1hat != train_smote[[response_col]]))
    Test_Error_glms <- c(Test_Error_glms, mean(y2hat != test_data[[response_col]]))
  }
  
  # Plot and evaluate errors
  plot(cutoff_values, Test_Error_glms, type = "b", col = "blue", pch = 19,
       xlab = "Cutoff Value", ylab = "Test Error", main = "Test Error vs Cutoff Values")
  
  # Return error values
  return(list(Train_Error_glms = Train_Error_glms, Test_Error_glms = Test_Error_glms))
}

# Apply Logistic Regression with Stepwise Selection and SMOTE for Data1 and Data2
logistic_regression_step_smote(Data1_train_10_processed, Data1_test_10_processed, response_col = "DUMMY_KILLED")
# Start:  AIC=802476
# as.factor(DUMMY_KILLED) ~ BOROUGH + ZIP_CODE + LATITUDE_CRASH + 
#   LONGITUDE_CRASH + PRCP + SNOW + SNWD + TMAX + TMIN + YEAR + 
#   MONTH + DAY + HOUR + RUSH_HOUR + SEASON + WEEKDAY + GROUPED_FACTOR_VEHICLE_1 + 
#   GROUPED_FACTOR_VEHICLE_2 + VEHICLE_1_TYPE_REGROUP + VEHICLE_2_TYPE_REGROUP
# 
# Df Deviance    AIC
# - DAY                       1   802434 802474
# - TMIN                      1   802436 802476
# <none>                          802434 802476
# - SNWD                      1   802444 802484
# - LATITUDE_CRASH            1   802486 802526
# - TMAX                      1   802509 802549
# - RUSH_HOUR                 1   802579 802619
# - ZIP_CODE                  1   802791 802831
# - VEHICLE_1_TYPE_REGROUP    1   802895 802935
# - PRCP                      1   803086 803126
# - SNOW                      1   803151 803191
# - WEEKDAY                   1   803203 803243
# - MONTH                     1   803291 803331
# - SEASON                    1   803611 803651
# - LONGITUDE_CRASH           1   804173 804213
# - BOROUGH                   1   804672 804712
# - GROUPED_FACTOR_VEHICLE_2  1   805618 805658
# - GROUPED_FACTOR_VEHICLE_1  1   806363 806403
# - HOUR                      1   807117 807157
# - YEAR                      1   815625 815665
# - VEHICLE_2_TYPE_REGROUP    1   839858 839898
# 
# Step:  AIC=802474.4
# as.factor(DUMMY_KILLED) ~ BOROUGH + ZIP_CODE + LATITUDE_CRASH + 
#   LONGITUDE_CRASH + PRCP + SNOW + SNWD + TMAX + TMIN + YEAR + 
#   MONTH + HOUR + RUSH_HOUR + SEASON + WEEKDAY + GROUPED_FACTOR_VEHICLE_1 + 
#   GROUPED_FACTOR_VEHICLE_2 + VEHICLE_1_TYPE_REGROUP + VEHICLE_2_TYPE_REGROUP
# 
# Df Deviance    AIC
# - TMIN                      1   802436 802474
# <none>                          802434 802474
# + DAY                       1   802434 802476
# - SNWD                      1   802445 802483
# - LATITUDE_CRASH            1   802487 802525
# - TMAX                      1   802510 802548
# - RUSH_HOUR                 1   802580 802618
# - ZIP_CODE                  1   802791 802829
# - VEHICLE_1_TYPE_REGROUP    1   802896 802934
# - PRCP                      1   803086 803124
# - SNOW                      1   803152 803190
# - WEEKDAY                   1   803204 803242
# - MONTH                     1   803291 803329
# - SEASON                    1   803613 803651
# - LONGITUDE_CRASH           1   804174 804212
# - BOROUGH                   1   804672 804710
# - GROUPED_FACTOR_VEHICLE_2  1   805618 805656
# - GROUPED_FACTOR_VEHICLE_1  1   806363 806401
# - HOUR                      1   807117 807155
# - YEAR                      1   815626 815664
# - VEHICLE_2_TYPE_REGROUP    1   839860 839898
# 
# Step:  AIC=802474.2
# as.factor(DUMMY_KILLED) ~ BOROUGH + ZIP_CODE + LATITUDE_CRASH + 
#   LONGITUDE_CRASH + PRCP + SNOW + SNWD + TMAX + YEAR + MONTH + 
#   HOUR + RUSH_HOUR + SEASON + WEEKDAY + GROUPED_FACTOR_VEHICLE_1 + 
#   GROUPED_FACTOR_VEHICLE_2 + VEHICLE_1_TYPE_REGROUP + VEHICLE_2_TYPE_REGROUP
# 
# Df Deviance    AIC
# <none>                          802436 802474
# + TMIN                      1   802434 802474
# + DAY                       1   802436 802476
# - SNWD                      1   802447 802483
# - LATITUDE_CRASH            1   802489 802525
# - RUSH_HOUR                 1   802581 802617
# - ZIP_CODE                  1   802793 802829
# - VEHICLE_1_TYPE_REGROUP    1   802898 802934
# - PRCP                      1   803093 803129
# - SNOW                      1   803158 803194
# - WEEKDAY                   1   803206 803242
# - MONTH                     1   803398 803434
# - SEASON                    1   803769 803805
# - LONGITUDE_CRASH           1   804176 804212
# - TMAX                      1   804379 804415
# - BOROUGH                   1   804673 804709
# - GROUPED_FACTOR_VEHICLE_2  1   805621 805657
# - GROUPED_FACTOR_VEHICLE_1  1   806366 806402
# - HOUR                      1   807125 807161
# - YEAR                      1   815665 815701
# - VEHICLE_2_TYPE_REGROUP    1   839861 839897
# $Train_Error_glms
# [1] 0.3310309 0.4447011 0.4959622 0.5003736 0.4998578 0.4998009 0.4998009 0.4998009 0.4998009
# 
# $Test_Error_glms
# [1] 0.345360356 0.069950620 0.011633191 0.002726529 0.001363265 0.001302675 0.001302675
# [8] 0.001302675 0.001302675

logistic_regression_step_smote(Data2_train_10_processed, Data2_test_10_processed, response_col = "DUMMY_KILLED")
# Start:  AIC=803090
# as.factor(DUMMY_KILLED) ~ BOROUGH + RUSH_HOUR + YEAR + VEHICLE_1_TYPE_REGROUP + 
#   VEHICLE_2_TYPE_REGROUP + LATITUDE_CRASH + GROUPED_FACTOR_VEHICLE_1 + 
#   SNOW + SNWD + PRCP + TMIN
# 
# Df Deviance    AIC
# <none>                          803066 803090
# - SNWD                      1   803225 803247
# - LATITUDE_CRASH            1   803367 803389
# - PRCP                      1   803444 803466
# - VEHICLE_1_TYPE_REGROUP    1   803687 803709
# - SNOW                      1   803983 804005
# - RUSH_HOUR                 1   804147 804169
# - TMIN                      1   804484 804506
# - BOROUGH                   1   806271 806293
# - GROUPED_FACTOR_VEHICLE_1  1   810793 810815
# - YEAR                      1   817792 817814
# - VEHICLE_2_TYPE_REGROUP    1   848748 848770
# $Train_Error_glms
# [1] 0.3268508 0.4511306 0.4954918 0.4998502 0.4998009 0.4998009 0.4998009 0.4998009 0.4998009
# 
# $Test_Error_glms
# [1] 0.362567785 0.067496743 0.006786028 0.001423854 0.001332970 0.001302675 0.001302675
# [8] 0.001302675 0.001302675

## -----------------------------Monte Carlo Cross-Validation--------------------------------
# ------------------------------------------------------------------------------------------

# Set parameters
B <- 100  # Number of loops
set.seed(6242)  # Set the seed for randomization

# Initialize error storage
TRERR1 <- NULL  # Average training errors for dataset 1
TEALL1 <- NULL  # Final test errors for dataset 1
TRERR2 <- NULL  # Average training errors for dataset 2
TEALL2 <- NULL  # Final test errors for dataset 2

for (b in 1:B) {
  
  # Randomly select a set of indices for the test and train split
  flag <- sort(sample(1:n, n1))
  
  # Split data into train and test sets
  Data1_train <- Data1[-flag, ]
  Data1_test <- Data1[flag, ]
  Data2_train <- Data2[-flag, ]
  Data2_test <- Data2[flag, ]
  
  # Define response variables for both datasets
  y1 <- Data1$Response
  y2 <- Data2$Response
  
  # Initialize error vectors
  TrainErr1_CV <- NULL
  TestErr1_CV <- NULL
  TrainErr2_CV <- NULL
  TestErr2_CV <- NULL
  
  # Logistic Regression for Dataset 1
  mod_glm1_CV <- glm(Response ~ ., family = binomial, data = Data1_train)
  c <- 0.5  # Cutoff value
  TrainErr1_CV <- c(TrainErr1_CV, mean(ifelse(predict(mod_glm1_CV, Data1_train[, -c(1, 2)], type = 'response') > c, 1, 0) != y1))
  TestErr1_CV <- c(TestErr1_CV, mean(ifelse(predict(mod_glm1_CV, Data1_test[, -c(1, 2)], type = 'response') > c, 1, 0) != y2))
  
  # Logistic Regression for Dataset 2
  mod_glm2_CV <- glm(Response ~ ., family = binomial, data = Data2_train)
  TrainErr2_CV <- c(TrainErr2_CV, mean(ifelse(predict(mod_glm2_CV, Data2_train[, -c(1, 2)], type = 'response') > c, 1, 0) != y1))
  TestErr2_CV <- c(TestErr2_CV, mean(ifelse(predict(mod_glm2_CV, Data2_test[, -c(1, 2)], type = 'response') > c, 1, 0) != y2))
  
  # Logistic Regression with step() for Dataset 1
  mod_glms1_CV <- step(glm(Response ~ ., family = binomial, data = Data1_train))
  TrainErr1_CV <- c(TrainErr1_CV, mean(ifelse(predict(mod_glms1_CV, Data1_train[, -c(1, 2)], type = 'response') > c, 1, 0) != y1))
  TestErr1_CV <- c(TestErr1_CV, mean(ifelse(predict(mod_glms1_CV, Data1_test[, -c(1, 2)], type = 'response') > c, 1, 0) != y2))
  
  # Logistic Regression with step() for Dataset 2
  mod_glms2_CV <- step(glm(Response ~ ., family = binomial, data = Data2_train))
  TrainErr2_CV <- c(TrainErr2_CV, mean(ifelse(predict(mod_glms2_CV, Data2_train[, -c(1, 2)], type = 'response') > c, 1, 0) != y1))
  TestErr2_CV <- c(TestErr2_CV, mean(ifelse(predict(mod_glms2_CV, Data2_test[, -c(1, 2)], type = 'response') > c, 1, 0) != y2))
  
  # Store results for each iteration
  TRERR1 <- rbind(TRERR1, TrainErr1_CV)
  TEALL1 <- rbind(TEALL1, TestErr1_CV)
  TRERR2 <- rbind(TRERR2, TrainErr2_CV)
  TEALL2 <- rbind(TEALL2, TestErr2_CV)
}

# Results summary
dim(TEALL1)  # 100 5
dim(TEALL2)  # 100 5

colnames(TEALL1) <- c("Logistic Regression", "Logistic Regression 2")
colnames(TEALL2) <- c("Logistic Regression", "Logistic Regression 2")

# CV result statistics
mean_TRERR1 <- apply(TRERR1, 2, mean)
mean_TEALL1 <- apply(TEALL1, 2, mean)
var_TEALL1 <- apply(TEALL1, 2, var)

mean_TRERR2 <- apply(TRERR2, 2, mean)
mean_TEALL2 <- apply(TEALL2, 2, mean)
var_TEALL2 <- apply(TEALL2, 2, var)

# Plot CV Error
k <- c(1, 2, 3, 4, 5)

# Plot for Dataset 1 and Dataset 2
plot(k, mean_TEALL1, xlab = 'Models', ylab = 'CV Error', main = "CV Error Plot: Dataset 1")
plot(k, mean_TEALL2, xlab = 'Models', ylab = 'CV Error', main = "CV Error Plot: Dataset 2")

# Combined plot
TEALL <- cbind(mean_TEALL1, mean_TEALL2)
matplot(k, TEALL, type = "l", lty = 1, col = c("red", "blue"), xlab = "k", ylab = "CV Error", main = "CV Error Plot")
legend("topright", legend = c("Dataset 1", "Dataset 2"), col = c("red", "blue"), lty = 1, cex = 0.5)

## -----------------------------------------------------------------------------
# ## Statistical difference testing on Dataset 1
# T1_1 <- t.test(TEALL1[,1], TEALL1[,2], paired = TRUE)
# T2_1 <- t.test(TEALL1[,1], TEALL1[,3], paired = TRUE)
# T3_1 <- t.test(TEALL1[,1], TEALL1[,4], paired = TRUE)
# T4_1 <- t.test(TEALL1[,1], TEALL1[,5], paired = TRUE)
# 
# W1_1 <- wilcox.test(TEALL1[,1], TEALL1[,2], paired = TRUE)
# W2_1 <- wilcox.test(TEALL1[,1], TEALL1[,3], paired = TRUE)
# W3_1 <- wilcox.test(TEALL1[,1], TEALL1[,4], paired = TRUE)
# W4_1 <- wilcox.test(TEALL1[,1], TEALL1[,5], paired = TRUE)
# 
# p_values_T1_1 <- T1_1$p.value
# p_values_T2_1 <- T2_1$p.value
# p_values_T3_1 <- T3_1$p.value
# p_values_T4_1 <- T4_1$p.value
# 
# p_values_W1_1 <- W1_1$p.value
# p_values_W2_1 <- W2_1$p.value
# p_values_W3_1 <- W3_1$p.value
# p_values_W4_1 <- W4_1$p.value
# 
# # Results
# c(p_values_T1_1, p_values_T2_1, p_values_T3_1, p_values_T4_1)
# c(p_values_W1_1, p_values_W2_1, p_values_W3_1, p_values_W4_1)
# 
# # Boxplot for Dataset 1
# boxplot(TEALL1)
# 
# ## Statistical difference testing on Dataset 2
# T1_2 <- t.test(TEALL2[,1], TEALL2[,2], paired = TRUE)
# T2_2 <- t.test(TEALL2[,1], TEALL2[,3], paired = TRUE)
# T3_2 <- t.test(TEALL2[,1], TEALL2[,4], paired = TRUE)
# T4_2 <- t.test(TEALL2[,1], TEALL2[,5], paired = TRUE)
# 
# W1_2 <- wilcox.test(TEALL2[,1], TEALL2[,2], paired = TRUE)
# W2_2 <- wilcox.test(TEALL2[,1], TEALL2[,3], paired = TRUE)
# W3_2 <- wilcox.test(TEALL2[,1], TEALL2[,4], paired = TRUE)
# W4_2 <- wilcox.test(TEALL2[,1], TEALL2[,5], paired = TRUE)
# 
# p_values_T1_2 <- T1_2$p.value
# p_values_T2_2 <- T2_2$p.value
# p_values_T3_2 <- T3_2$p.value
# p_values_T4_2 <- T4_2$p.value
# 
# p_values_W1_2 <- W1_2$p.value
# p_values_W2_2 <- W2_2$p.value
# p_values_W3_2 <- W3_2$p.value
# p_values_W4_2 <- W4_2$p.value
# 
# # Results
# c(p_values_T1_2, p_values_T2_2, p_values_T3_2, p_values_T4_2)
# c(p_values_W1_2, p_values_W2_2, p_values_W3_2, p_values_W4_2)
# 
# # Boxplot for Dataset 2
# boxplot(TEALL2)

#####################Random Forest###########################################
# --------------------------- Random Forest ---------------------------------
# 1. **Random Forest with SMOTE and Variable Importance**
# ----------------------------------Random Forest with Parameter Tuning-------------------------------------
library(randomForest)
library(caret)
library(e1071)
update.packages("smotefamily")
library(smotefamily) # For SMOTE
library(ROCR)

# 1. Prepare Data and Apply SMOTE from smotefamily
X_train_10 <- Data1_train_10_processed[, -which(names(Data1_train_10_processed) == "DUMMY_KILLED")]
y_train_10 <- Data1_train_10_processed$DUMMY_KILLED

# Apply SMOTE using smotefamily
smote_output <- SMOTE(X_train_10, y_train_10, K = 5) # K is the number of nearest neighbors
train_smote <- smote_output$data
train_smote$class <- as.factor(train_smote$class) # Ensure it's a factor
head(train_smote)

# 2. Set up cross-validation and parameter tuning grid for Random Forest
set.seed(111)

# Define tuning grid for Random Forest
tuneGrid <- expand.grid(.mtry = c(3, 5)) # Number of variables to be randomly sampled at each split
ntrees <- c(100, 200, 300)    
nodesize <- c(1, 5, 10)
param_grid <- expand.grid(ntrees = ntrees,
                          nodesize = nodesize)

# 3. Set up cross-validation with SMOTE resampling
cv_folds <- createFolds(train_smote$class, k = 5, returnTrain = TRUE)

ctrl <- trainControl(method = "cv",
                     number = 3,
                     search = 'grid',
                     classProbs = TRUE,
                     savePredictions = "final",
                     index = cv_folds,
                     summaryFunction = twoClassSummary)

# 4. Train Random Forest model with tuning using caret
# rf_tune <- train(
#   make.names(class) ~ .,  # Formula
#   data = train_smote,  # Training data
#   method = "rf",  # Random Forest method
#   trControl = ctrl,  # Cross-validation control
#   tuneGrid = tuneGrid,  # Parameter tuning grid
#   importance = TRUE,  # Calculate variable importance
#   metric = "ROC"  # Optimize for ROC AUC
# )

data_maxnode <- vector("list", nrow(param_grid))
for(i in 1:nrow(param_grid)){
  ntree <- param_grid[i,1]
  nodesize <- param_grid[i,2]
  set.seed(333)
  rf_model <- train(make.names(class)~.,
                    data = train_smote,
                    method = "rf",
                    importance=TRUE,
                    metric = "ROC",
                    trControl = ctrl,
                    tuneGrid = tuneGrid,
                    ntree = ntree,
                    nodesize = nodesize)
  data_maxnode[[i]] <- rf_model
}

names(data_maxnode) <- paste("ntrees:", param_grid$ntrees,
                              "nodesize:", param_grid$nodesize)

results_mtry <- resamples(data_maxnode)

summary(results_mtry)
# Call:
#   summary.resamples(object = results_mtry)
# 
# Models: ntrees: 100 nodesize: 1, ntrees: 200 nodesize: 1, ntrees: 300 nodesize: 1, ntrees: 100 nodesize: 5, ntrees: 200 nodesize: 5, ntrees: 300 nodesize: 5, ntrees: 100 nodesize: 10, ntrees: 200 nodesize: 10, ntrees: 300 nodesize: 10 
# Number of resamples: 5 
# 
# ROC 
# Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
# ntrees: 100 nodesize: 1  0.9999405 0.9999617 0.9999657 0.9999663 0.9999702 0.9999933    0
# ntrees: 200 nodesize: 1  0.9999699 0.9999753 0.9999773 0.9999803 0.9999853 0.9999939    0
# ntrees: 300 nodesize: 1  0.9999731 0.9999765 0.9999802 0.9999824 0.9999886 0.9999936    0
# ntrees: 100 nodesize: 5  0.9999227 0.9999609 0.9999638 0.9999619 0.9999703 0.9999919    0
# ntrees: 200 nodesize: 5  0.9999561 0.9999708 0.9999848 0.9999781 0.9999886 0.9999902    0
# ntrees: 300 nodesize: 5  0.9999670 0.9999765 0.9999881 0.9999827 0.9999903 0.9999913    0
# ntrees: 100 nodesize: 10 0.9998941 0.9999307 0.9999485 0.9999434 0.9999679 0.9999759    0
# ntrees: 200 nodesize: 10 0.9999529 0.9999577 0.9999689 0.9999712 0.9999879 0.9999886    0
# ntrees: 300 nodesize: 10 0.9999686 0.9999717 0.9999800 0.9999809 0.9999917 0.9999926    0
# 
# Sens 
#                          Min. 1st Qu. Median Mean 3rd Qu. Max. NA's
# ntrees: 100 nodesize: 1     1       1      1    1       1    1    0
# ntrees: 200 nodesize: 1     1       1      1    1       1    1    0
# ntrees: 300 nodesize: 1     1       1      1    1       1    1    0
# ntrees: 100 nodesize: 5     1       1      1    1       1    1    0
# ntrees: 200 nodesize: 5     1       1      1    1       1    1    0
# ntrees: 300 nodesize: 5     1       1      1    1       1    1    0
# ntrees: 100 nodesize: 10    1       1      1    1       1    1    0
# ntrees: 200 nodesize: 10    1       1      1    1       1    1    0
# ntrees: 300 nodesize: 10    1       1      1    1       1    1    0
# 
# Spec 
# Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
# ntrees: 100 nodesize: 1  0.9983303 0.9985201 0.9988237 0.9986719 0.9988237 0.9988616    0
# ntrees: 200 nodesize: 1  0.9982923 0.9985201 0.9988236 0.9986567 0.9988237 0.9988237    0
# ntrees: 300 nodesize: 1  0.9983303 0.9985201 0.9988236 0.9986643 0.9988237 0.9988237    0
# ntrees: 100 nodesize: 5  0.9981785 0.9982545 0.9986719 0.9985049 0.9986719 0.9987477    0
# ntrees: 200 nodesize: 5  0.9981406 0.9983304 0.9986719 0.9985277 0.9987477 0.9987478    0
# ntrees: 300 nodesize: 5  0.9981785 0.9983304 0.9986339 0.9985125 0.9987098 0.9987098    0
# ntrees: 100 nodesize: 10 0.9981027 0.9981406 0.9985580 0.9983986 0.9985580 0.9986339    0
# ntrees: 200 nodesize: 10 0.9980647 0.9981027 0.9985200 0.9983986 0.9986339 0.9986719    0
# ntrees: 300 nodesize: 10 0.9980647 0.9981406 0.9985959 0.9984214 0.9986339 0.9986719    0

# to get the best average performance for each model
lapply(data_maxnode, function(x) x$results[x$results$ROC == max(x$results$ROC),])
# $`ntrees: 100 nodesize: 1`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999663    1 0.9986719 1.893113e-05      0 0.0002354651
# 
# $`ntrees: 200 nodesize: 1`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999803    1 0.9986567 9.381088e-06      0 0.0002423985
# 
# $`ntrees: 300 nodesize: 1`
# mtry       ROC Sens      Spec        ROCSD SensSD      SpecSD
# 1    3 0.9999824    1 0.9986643 8.524244e-06      0 0.000228325
# 
# $`ntrees: 100 nodesize: 5`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999619    1 0.9985049 2.504672e-05      0 0.0002664466
# 
# $`ntrees: 200 nodesize: 5`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999781    1 0.9985277 1.449106e-05      0 0.0002767895
# 
# $`ntrees: 300 nodesize: 5`
# mtry       ROC Sens      Spec       ROCSD SensSD       SpecSD
# 1    3 0.9999827    1 0.9985125 1.05527e-05      0 0.0002435808
# 
# $`ntrees: 100 nodesize: 10`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999434    1 0.9983986 3.270358e-05      0 0.0002551262
# 
# $`ntrees: 200 nodesize: 10`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999712    1 0.9983986 1.660777e-05      0 0.0002932105
# 
# $`ntrees: 300 nodesize: 10`
# mtry       ROC Sens      Spec        ROCSD SensSD       SpecSD
# 1    3 0.9999809    1 0.9984214 1.106868e-05      0 0.0002934552

# View the best tuning parameters
# print(rf_tune$bestTune)  # Best combination of parameters

# 5. Model summary and performance evaluation
# print(rf_tune)  # Output results of cross-validation

# RF model with tuned parameters
rf_model <- randomForest(as.factor(class) ~., data=train_smote, ntree= 300, 
                      mtry=3, nodesize =5, importance=TRUE)
# Variable Importance
# varImpPlot(rf_tune$finalModel, main = "Variable Importance (Tuned Random Forest)")
varImpPlot(rf_model, main = "Variable Importance (Tuned Random Forest)")

# 6. Prediction on Test Data
rf.pred <- predict(rf_model, Data1_test_10_processed, type = "prob")[, 2] # Get probabilities for the positive class
rf.pred_class <- predict(rf_model, Data1_test_10_processed, type = "class")

# Confusion Matrix and Performance Evaluation for Random Forest
conf_matrix_rf <- confusionMatrix(as.factor(rf.pred_class), as.factor(Data1_test_10_processed$DUMMY_KILLED))
print(conf_matrix_rf)
# Confusion Matrix and Statistics
# 
# Reference
# Prediction     0     1
# 0 32966    43
# 1     0     0
# 
# Accuracy : 0.9987          
# 95% CI : (0.9982, 0.9991)
# No Information Rate : 0.9987          
# P-Value [Acc > NIR] : 0.5404          
# 
# Kappa : 0               
# 
# Mcnemar's Test P-Value : 1.504e-10       
#                                           
#             Sensitivity : 1.0000          
#             Specificity : 0.0000          
#          Pos Pred Value : 0.9987          
#          Neg Pred Value :    NaN          
#              Prevalence : 0.9987          
#          Detection Rate : 0.9987          
#    Detection Prevalence : 1.0000          
#       Balanced Accuracy : 0.5000          
#                                           
#        'Positive' Class : 0        

# ROC Curve and AUC for Random Forest
prediction_rf <- prediction(rf.pred, Data1_test_10_processed$DUMMY_KILLED)
perf_rf <- performance(prediction_rf, "tpr", "fpr")
plot(perf_rf, main = "ROC Curve (Tuned Random Forest)")
abline(a = 0, b = 1, lty = 2) # Diagonal line for reference
auc_rf <- performance(prediction_rf, "auc")@y.values[[1]]
cat("AUC on Test Data (Tuned Random Forest):", round(auc_rf, 4), "\n")
# AUC on Test Data (Tuned Random Forest): 0.5662

##--------------------F1 score as metrics-----------------------
# Define custom trainControl using F1 score
ctrl_F1 <- trainControl(method = "cv",
                     number = 3,
                     search = 'grid',
                     classProbs = TRUE,
                     savePredictions = "final",
                     index = cv_folds,
                     summaryFunction = function(data, lev = NULL, model = NULL) {
                       predicted <- data$pred
                       actual <- data$obs
                       f1 <- F1_Score(predicted, actual)
                       return(c(F1 = f1))
                     },
                     verboseIter = TRUE)

# Random Forest Parameter Tuning
data_maxnode_F1 <- vector("list", nrow(param_grid))
for(i in 1:nrow(param_grid)){
  ntree <- param_grid[i,1]
  nodesize <- param_grid[i,2]
  set.seed(333)
  
  # Train Random Forest model with F1 score as the metric
  rf_model_F1 <- train(make.names(class) ~ ., 
                    data = train_smote,
                    method = "rf",
                    importance = TRUE,
                    metric = "F1",  # Specify F1 as the evaluation metric
                    trControl = ctrl_F1,
                    tuneGrid = tuneGrid,
                    ntree = ntree,
                    nodesize = nodesize)
  
  # Store the model
  data_maxnode_F1[[i]] <- rf_model_F1
}

# Assign model names based on parameters
names(data_maxnode_F1) <- paste("ntrees:", param_grid$ntrees,
                             "nodesize:", param_grid$nodesize)

# 5. Evaluate results
results_mtry_F1 <- resamples(data_maxnode_F1)
summary(results_mtry_F1)
# 
# + Fold1: mtry=3 
# - Fold1: mtry=3 
# + Fold1: mtry=5 
# - Fold1: mtry=5 
# + Fold2: mtry=3 
# - Fold2: mtry=3 
# + Fold2: mtry=5 
# - Fold2: mtry=5 
# + Fold3: mtry=3 
# - Fold3: mtry=3 
# + Fold3: mtry=5 
# - Fold3: mtry=5 
# + Fold4: mtry=3 
# - Fold4: mtry=3 
# + Fold4: mtry=5 
# - Fold4: mtry=5 
# + Fold5: mtry=3 
# - Fold5: mtry=3 
# + Fold5: mtry=5 
# - Fold5: mtry=5 
# Aggregating results
# Something is wrong; all the F1 metric values are missing:
#   F1     
# Min.   : NA  
# 1st Qu.: NA  
# Median : NA  
# Mean   :NaN  
# 3rd Qu.: NA  
# Max.   : NA  
# NA's   :2    
# Error: Stopping
# In addition: Warning message:
# In nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo,  :
#   There were missing values in resampled performance measures.


#####################Boosting (GBM Model)###########################################
# --------------------------- Boosting (GBM Model) ---------------------------------
# Define parameter grid for tuning
gbmGrid <- expand.grid(
  n.trees = c(50, 100, 500), 
  interaction.depth = c(1, 3, 5), 
  shrinkage = c(0.01, 0.1), 
  n.minobsinnode = c(1, 5, 10)
)

fitControl <- trainControl(
  method = "cv",
  number = 3,
  classProbs = TRUE,
  sampling = "smote", # Apply SMOTE resampling
  summaryFunction = twoClassSummary # Important for AUC
)

set.seed(222)
# Fit Boosting model using caret
gbmFit <- train(
  make.names(class) ~ ., 
  data = train_smote, 
  method = "gbm", 
  trControl = fitControl, 
  tuneGrid = gbmGrid, 
  metric = "ROC", 
  verbose = FALSE
)


# Print Boosting results
print(gbmFit)
plot(gbmFit)

# Stochastic Gradient Boosting 
# 
# 263631 samples
# 20 predictor
# 2 classes: 'X0', 'X1' 
# 
# No pre-processing
# Resampling: Cross-Validated (3 fold) 
# Summary of sample sizes: 175754, 175754, 175754 
# Addtional sampling using SMOTE
# 
# Resampling results across tuning parameters:
#   
#   shrinkage  interaction.depth  n.minobsinnode  n.trees  ROC        Sens       Spec     
# 0.01       1                   1               50      0.7419044  0.5348000  0.9449238
# 0.01       1                   1              100      0.8440745  0.5348000  0.9449390
# 0.01       1                   1              500      0.9312761  0.7731974  0.9045559
# 0.01       1                   5               50      0.7419044  0.5348000  0.9449390
# 0.01       1                   5              100      0.8519786  0.5348000  0.9449390
# 0.01       1                   5              500      0.9304077  0.7690190  0.9112118
# 0.01       1                  10               50      0.7419044  0.5348000  0.9449238
# 0.01       1                  10              100      0.8440715  0.5348000  0.9449238
# 0.01       1                  10              500      0.9311933  0.7671156  0.9105060
# 0.01       3                   1               50      0.8783150  0.7625883  0.8270455
# 0.01       3                   1              100      0.9363616  0.7633391  0.8828047
# 0.01       3                   1              500      0.9916992  0.9671945  0.9504869
# 0.01       3                   5               50      0.8783164  0.7625883  0.8270379
# 0.01       3                   5              100      0.9360081  0.7638775  0.8827971
# 0.01       3                   5              500      0.9916080  0.9669139  0.9519971
# 0.01       3                  10               50      0.8783150  0.7625883  0.8270379
# 0.01       3                  10              100      0.9353050  0.7644690  0.8827896
# 0.01       3                  10              500      0.9917287  0.9666636  0.9516860
# 0.01       5                   1               50      0.9161816  0.9212015  0.7879223
# 0.01       5                   1              100      0.9511398  0.9094397  0.8778413
# 0.01       5                   1              500      0.9979008  0.9881472  0.9717675
# 0.01       5                   5               50      0.9188938  0.9212015  0.7879223
# 0.01       5                   5              100      0.9520559  0.9094018  0.8778337
# 0.01       5                   5              500      0.9979252  0.9884051  0.9719117
# 0.01       5                  10               50      0.9182053  0.9212015  0.7879071
# 0.01       5                  10              100      0.9511471  0.9100009  0.8778413
# 0.01       5                  10              500      0.9979405  0.9884961  0.9738166
# 0.10       1                   1               50      0.9288928  0.7781797  0.9140958
# 0.10       1                   1              100      0.9593495  0.8480753  0.9102100
# 0.10       1                   1              500      0.9913700  0.9702354  0.9546838
# 0.10       1                   5               50      0.9299191  0.7782631  0.9097167
# 0.10       1                   5              100      0.9594490  0.8508736  0.9089274
# 0.10       1                   5              500      0.9913177  0.9704022  0.9549722
# 0.10       1                  10               50      0.9290828  0.7803789  0.9107792
# 0.10       1                  10              100      0.9597144  0.8493190  0.9110828
# 0.10       1                  10              500      0.9914226  0.9707283  0.9542436
# 0.10       3                   1               50      0.9915706  0.9673461  0.9464417
# 0.10       3                   1              100      0.9979329  0.9878742  0.9691719
# 0.10       3                   1              500      0.9997049  0.9999924  0.9986719
# 0.10       3                   5               50      0.9916378  0.9677177  0.9492574
# 0.10       3                   5              100      0.9979328  0.9883975  0.9680107
# 0.10       3                   5              500      0.9997072  1.0000000  0.9986794
# 0.10       3                  10               50      0.9913697  0.9657915  0.9478761
# 0.10       3                  10              100      0.9979569  0.9878970  0.9694527
# 0.10       3                  10              500      0.9997026  1.0000000  0.9986870
# 0.10       5                   1               50      0.9976952  0.9882231  0.9716764
# 0.10       5                   1              100      0.9996117  0.9984530  0.9915454
# 0.10       5                   1              500      0.9995530  0.9996587  0.9987022
# 0.10       5                   5               50      0.9980666  0.9889814  0.9734220
# 0.10       5                   5              100      0.9996068  0.9985895  0.9912494
# 0.10       5                   5              500      0.9997337  0.9998332  0.9986719
# 0.10       5                  10               50      0.9978561  0.9887008  0.9712059
# 0.10       5                  10              100      0.9996029  0.9983923  0.9913405
# 0.10       5                  10              500      0.9997468  0.9999242  0.9986946
# 
# ROC was used to select the optimal model using the largest value.
# The final values used for the model were n.trees = 500, interaction.depth = 5, shrinkage =
#   0.1 and n.minobsinnode = 10.

# optimal tune values
gbmFit$finalModel$tuneValue
# n.trees interaction.depth shrinkage n.minobsinnode
# 54     500                 5       0.1             10

whichTwoPct <- tolerance(gbmFit$results, metric = "ROC", 
                         tol = 2, maximize = TRUE)  
cat("best model within 2 pct of best:")
gbmFit$results[whichTwoPct,1:6]
# shrinkage interaction.depth n.minobsinnode n.trees       ROC      Sens
# 37       0.1                 3              1      50 0.9915706 0.9673461

# optimal model
gbmFit_optimal <- gbm(
  class ~ ., 
  data = train_smote, 
  distribution = 'bernoulli',
  n.trees = 50,  # Optimal n.trees
  shrinkage = 0.1,  # Optimal shrinkage
  interaction.depth = 3,  # Optimal interaction depth
  n.minobsinnode = 1,  # Optimal minobsinnode
  cv.folds = 10  # Cross-validation to assess the model's generalization
)

summary(gbmFit_optimal)

## Training error
predgbm <- predict(gbm_model,
                   newdata = train_smote, 
                   n.trees=50, 
                   shrinkage = 0.1,
                   interaction.depth = 3,
                   n.minobsinnode = 1,
                   type="response")
y1hat <- ifelse(predgbm < 0.5, 0, 1)
sum(y1hat != y1)/length(y1)  ##Training error = 0.09162896

## Testing Error
y2hat <- ifelse(predict(gbm_model,newdata = Data1_test_10_processed[,-1], n.trees=50, type="response") < 0.5, 0, 1)
mean(y2hat != y2) 
## Testing error = 0.1357466

# Now compute the confusion matrix
conf_matrix_gbm <- confusionMatrix(gbm_pred_class, factor(Data1_test_10_processed$DUMMY_KILLED, levels = c(0, 1)))

# Print the confusion matrix
print(conf_matrix_gbm)


# ROC Curve and AUC for Boosting
prediction_gbm <- prediction(gbm_pred, Data1_test_processed$DUMMY_KILLED)
perf_gbm <- performance(prediction_gbm, "tpr", "fpr")
plot(perf_gbm, main = "ROC Curve (Boosting)")
abline(a = 0, b = 1, lty = 2) # Diagonal line for reference
auc_gbm <- performance(prediction_gbm, "auc")@y.values[[1]]
cat("AUC on Test Data (Boosting):", round(auc_gbm, 4), "\n")
# AUC on Test Data (Boosting): 0.6813 

# 3. **Variable Importance for Boosting**
# Variable importance from the trained GBM model
importance_gbm <- varImp(gbmFit, scale = FALSE)
print(importance_gbm)

# Plot variable importance for Boosting
plot(importance_gbm, main = "Variable Importance (Boosting)")

# Optional: You can adjust the importance plot parameters (like top variables) if needed


#####################SVM###########################################
# --------------------------- SVM ---------------------------------
# Tune SVM hyperparameters (cost and gamma)
tune_out <- tune.svm(
  x = Data1_train_processed[, -which(names(Data1_train_processed) == "DUMMY_KILLED")],
  y = Data1_train_processed$DUMMY_KILLED,
  gamma = 10^(-3:3),
  cost = c(0.01, 0.1, 1, 10, 100, 1000),
  kernel = "radial"
)

# Fit SVM model with best parameters
svm_model <- svm(
  as.factor(class) ~ ., 
  data = train_smote, 
  method = "C-classification", 
  kernel = "radial", 
  cost = tune_out$best.parameters$cost, 
  gamma = tune_out$best.parameters$gamma
)

# Predict on test data
svm_pred <- predict(svm_model, Data1_test_10_processed, type = "response")
svm_pred_class <- predict(svm_model, Data1_test_10_processed, type = "class")

# Confusion Matrix and Performance Evaluation for SVM
conf_matrix_svm <- confusionMatrix(svm_pred_class, Data1_test_10_processed$DUMMY_KILLED)
print(conf_matrix_svm)

# ROC Curve and AUC for SVM
prediction_svm <- prediction(as.numeric(svm_pred_class), Data1_test_10_processed$DUMMY_KILLED)
perf_svm <- performance(prediction_svm, "tpr", "fpr")
plot(perf_svm, main = "ROC Curve (SVM)")
abline(a = 0, b = 1, lty = 2) # Diagonal line for reference
auc_svm <- performance(prediction_svm, "auc")@y.values[[1]]
cat("AUC on Test Data (SVM):", round(auc_svm, 4), "\n")


# Feature Importance for SVM 
M <- fit(DUMMY_KILLED ~ ., data = Data1_train_10_processed, model = "svm", kpar = list(sigma = tune_out$best.parameters$gamma), C = tune_out$best.parameters$cost)
svm_imp <- Importance(M, data = Data1_train_10_processed)
print(svm_imp)
